# Applied Graph Algorithms

In [92]:
from neo4j.v1 import GraphDatabase, basic_auth
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd

In [93]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo"))

## Betweenness Centrality

Betweenness centrality identifies nodes that are strategically positioned in the network, meaning that information will often travel through that person. Such an intermediary position gives that person power and influence.

Betweenness centrality is a raw count of the number of short paths that go through a given node. For example, if a node is located on a bottleneck between two large communities, then it will have high betweenness.

In [94]:
query = """\
CALL algo.betweenness.stream("Character", "INTERACTS1")
YIELD nodeId, centrality
MATCH (c:Character) WHERE ID(c) = nodeId
RETURN c.name, centrality
"""

with driver.session() as session:
    result = session.run(query)
df = pd.DataFrame([dict(record) for record in result])    

In [95]:
df.sort_values(by=["centrality"], ascending=False).head()

,c.name,centrality
750,Tyrion-Lannister,2682.658333
742,Sansa-Stark,2495.428571
736,Robb-Stark,1296.169048
656,Eddard-Stark,1058.450000
751,Tywin-Lannister,929.833333


In [96]:
query = """\
CALL algo.betweenness.stream("Character", "INTERACTS1")
YIELD nodeId, centrality
MATCH (c:Character) WHERE ID(c) = nodeId
WITH c, centrality, [(c)-[r:INTERACTS1]-(other) | {character: other.name, weight: r.weight}] AS interactions
RETURN c.name, centrality, apoc.coll.sum([i in interactions | i.weight]) AS totalInteractions
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])  

In [97]:
df.sort_values(by=["centrality"], ascending=False).head(10)

,c.name,centrality,totalInteractions
750,Tyrion-Lannister,2682.658333,650.0
742,Sansa-Stark,2495.428571,545.0
736,Robb-Stark,1296.169048,516.0
656,Eddard-Stark,1058.450000,1284.0
751,Tywin-Lannister,929.833333,181.0
738,Robert-Baratheon,792.791667,941.0
693,Jon-Snow,790.166667,784.0
690,Joffrey-Baratheon,537.450000,422.0
732,Renly-Baratheon,446.308333,186.0
683,Jaime-Lannister,389.119048,241.0


In [98]:
df.sort_values(by=["totalInteractions"], ascending=False).head(10)

,c.name,centrality,totalInteractions
656,Eddard-Stark,1058.450000,1284.0
738,Robert-Baratheon,792.791667,941.0
693,Jon-Snow,790.166667,784.0
750,Tyrion-Lannister,2682.658333,650.0
742,Sansa-Stark,2495.428571,545.0
632,Bran-Stark,22.500000,531.0
636,Catelyn-Stark,172.042857,520.0
736,Robb-Stark,1296.169048,516.0
646,Daenerys-Targaryen,160.005952,443.0
623,Arya-Stark,0.000000,430.0


## Storing betweenness centrality

Although the betweenness centrality algorithm runs very quickly on this dataset we wouldn’t usually be running this types of algorithms in the normal request/response flow of a web/mobile app. Instead of that we can store the result of the calculation as a property on the node and then refer to it in future queries.

Each of the algorithms has a variant that saves its output to the database rather than returning a stream. Let’s run the betweenness centrality algorithm and store the result as a property named `book1BetweennessCentrality`:

In [99]:
query = """\
CALL algo.betweenness("Character", "INTERACTS1", {writeProperty: "book1BetweennessCentrality"})
"""
with driver.session() as session:
    session.run(query)

We can write the following query to find the most influential characters:

In [100]:
query = """\
MATCH (c:Character)
RETURN c.name, c.book1BetweennessCentrality AS centrality
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])  

In [101]:
df.sort_values(by=["centrality"], ascending=False).head()

,c.name,centrality
750,Tyrion-Lannister,2682.658333
742,Sansa-Stark,2495.428571
736,Robb-Stark,1296.169048
656,Eddard-Stark,1058.450000
751,Tywin-Lannister,929.833333


## Exercise: Betweenness Centrality for books 2-5

Now we want to calculate the betweenness centrality for the other books in the series and store the results in the database.

* Write queries that call algo.betweenness for the INTERACTS2, INTERACTS3, and INTERACTS45 relationship types.

After you’ve done that see if you can write queries to answer the following questions:

* Which character had the biggest increase in influence from book 1 to 5?

Wh* ich character had the biggest decrease?

Bonus question:

* Which characters who were in the top 10 influencers in book 1 are also in the top 10 influencers in book 5?

## Page Rank

This is another version of weighted degree centrality with a feedback loop. This time, you only get your “fair share” of your neighbor’s importance.

i.e. your neighbor’s importance is split between their neighbors, proportional to the number of interactions with that neighbor.

Intuitively, PageRank captures how effectively you are taking advantage of your network contacts. In our context, PageRank centrality nicely captures narrative tension. Indeed, major developments occur when two important characters interact.

In [102]:
with driver.session() as session:
    result = session.run("CALL algo.pageRank('Character', 'INTERACTS1', {writeProperty:'book1PageRank'})")
    print(result.peek())
    
    result = session.run("CALL algo.pageRank('Character', 'INTERACTS2', {writeProperty:'book2PageRank'})")
    print(result.peek()) 
    
    result = session.run("CALL algo.pageRank('Character', 'INTERACTS3', {writeProperty:'book3PageRank'})")
    print(result.peek())     
    
    result = session.run("CALL algo.pageRank('Character', 'INTERACTS45', {writeProperty:'book45PageRank'})")
    print(result.peek())         

<Record nodes=796 iterations=20 loadMillis=1 computeMillis=0 writeMillis=1 dampingFactor=0.85 write=True writeProperty='book1PageRank'>
<Record nodes=796 iterations=20 loadMillis=1 computeMillis=0 writeMillis=1 dampingFactor=0.85 write=True writeProperty='book2PageRank'>
<Record nodes=796 iterations=20 loadMillis=1 computeMillis=0 writeMillis=0 dampingFactor=0.85 write=True writeProperty='book3PageRank'>
<Record nodes=796 iterations=20 loadMillis=2 computeMillis=0 writeMillis=0 dampingFactor=0.85 write=True writeProperty='book45PageRank'>


In [103]:
query = """\
MATCH (c:Character)
WITH c, [(c)-[r:INTERACTS1]-(other) | {character: other.name, weight: r.weight}] AS interactions
RETURN c.name, c.book1PageRank AS pageRank, c.book1BetweennessCentrality as centrality, 
       apoc.coll.sum([i in interactions | i.weight]) AS totalInteractions
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])      

In [104]:
df.sort_values(by=["centrality"], ascending = False).head(10)

,c.name,centrality,pageRank,totalInteractions
750,Tyrion-Lannister,2682.658333,4.369366,650.0
742,Sansa-Stark,2495.428571,1.932841,545.0
736,Robb-Stark,1296.169048,1.301130,516.0
656,Eddard-Stark,1058.450000,0.660161,1284.0
751,Tywin-Lannister,929.833333,2.983815,181.0
738,Robert-Baratheon,792.791667,2.074205,941.0
693,Jon-Snow,790.166667,1.187094,784.0
690,Joffrey-Baratheon,537.450000,0.443956,422.0
732,Renly-Baratheon,446.308333,0.478287,186.0
683,Jaime-Lannister,389.119048,0.472507,241.0


In [105]:
df.sort_values(by=["pageRank"], ascending = False).head(10)

,c.name,centrality,pageRank,totalInteractions
750,Tyrion-Lannister,2682.658333,4.369366,650.0
760,Varys,0.000000,3.544373,231.0
751,Tywin-Lannister,929.833333,2.983815,181.0
738,Robert-Baratheon,792.791667,2.074205,941.0
742,Sansa-Stark,2495.428571,1.932841,545.0
769,Walder-Frey,0.000000,1.883626,41.0
736,Robb-Stark,1296.169048,1.301130,516.0
767,Willis-Wode,0.000000,1.209874,28.0
693,Jon-Snow,790.166667,1.187094,784.0
766,Vardis-Egen,0.000000,1.181390,37.0


You’ll notice that there are some characters who have a high page rank but a very low betweenness centrality score.

This suggests that they aren’t necessarily influential in their own right, but are friends with important people. Varys is a good example of a character that fits this profile.

## Community Detection

We can detect communities in our data by running an algorithm which traverses the graph structure to find highly connected subgraphs with fewer connections other other subgraphs.

Run the following query to calculate the communities that exist based on interactions across all the books.

In [106]:
query = """\
CALL algo.labelPropagation(
  'MATCH (c:Character) RETURN id(c) as id',
  'MATCH (c:Character)--->(c2) RETURN id(c) as source, id(c2) as target, SUM(rel.weight) as weight',
  'OUTGOING',
  {graph:'cypher', partitionProperty: 'community', iterations: 10})
"""

with driver.session() as session:
    result = session.run(query)
    print(result.peek())

<Record nodes=19 iterations=10 loadMillis=5 computeMillis=4 writeMillis=0 write=True weightProperty='weight' partitionProperty='community'>


In [107]:
query = """\
MATCH (c:Character)
WHERE exists(c.community)
RETURN c.community AS community, count(*) AS count
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])   

In [108]:
df.sort_values(by=["count"], ascending = False).head(10)

,community,count
55,760.0,321
32,136.0,45
0,137.0,18
24,561.0,18
67,556.0,16
52,569.0,11
66,143.0,10
33,755.0,8
20,301.0,8
40,276.0,7


## Querying Communities

It’d be good to know who are the influential people in each community. To do that we’ll need to calculate a Page Rank score for each character across all the books:

In [109]:
query = """\
CALL algo.pageRank(
  'MATCH (c:Character) RETURN id(c) as id',
  'MATCH (c:Character)-[rel]->(c2) RETURN id(c) as source,id(c2) as target, SUM(rel.weight) as weight',
  {graph:'cypher', writeProperty: 'pageRank', iterations: 10})
"""

with driver.session() as session:
    result = session.run(query)
    print(result.peek())

<Record nodes=796 iterations=10 loadMillis=15 computeMillis=0 writeMillis=0 dampingFactor=0.85 write=True writeProperty='pageRank'>


In [110]:
query = """\
MATCH (c:Character)
WHERE exists(c.community)
WITH c ORDER BY c.pageRank DESC
RETURN c.community as cluster, count(*) AS count, collect(c.name)[0] AS mostInfluential
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])   

In [111]:
df.sort_values(by=["count"], ascending = False).head(10)

,cluster,count,mostInfluential
55,760.0,321,Tyrion-Lannister
32,136.0,45,Theon-Greyjoy
0,137.0,18,Tormund
26,561.0,18,Daenerys-Targaryen
67,556.0,16,Victarion-Greyjoy
50,569.0,11,Quentyn-Martell
68,143.0,10,Theomore
33,755.0,8,Quaro
20,301.0,8,Satin
25,570.0,7,Garin-(orphan)


## Intra-community Page Rank

We can also calculate the Page Rank within communities.

Run the following query to calculate the page rank for the 2nd largest community:

In [112]:
query = """\
MATCH (c:Character) WHERE EXISTS(c.community)
WITH c.community AS communityId, COUNT(*) AS count
ORDER BY count DESC
SKIP 1 LIMIT 1
CALL apoc.cypher.doIt(
  "CALL algo.pageRank(
    'MATCH (c:Character) WHERE c.community =" + communityId + " RETURN id(c) as id',
    'MATCH (c:Character)-[rel]->(c2) WHERE c.community =" + communityId + " AND c2.community =" + communityId + " RETURN id(c) as source,id(c2) as target, sum(rel.weight) as weight',
    {graph:'cypher', writeProperty: 'communityPageRank'}) YIELD nodes RETURN count(*)", {})
YIELD value
RETURN value
"""

with driver.session() as session:
    result = session.run(query)
    print(result.peek())

<Record value={'count(*)': 1}>


In [113]:
query = """\
MATCH (c:Character) WHERE EXISTS(c.community)
WITH c.community AS communityId, COUNT(*) AS count
ORDER BY count DESC
SKIP 1 LIMIT 1
MATCH (c:Character) WHERE c.community = communityId
RETURN c.name AS character, c.communityPageRank as pageRank
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])       

In [114]:
df.sort_values(by=["pageRank"], ascending = False).head(10)

,character,pageRank
44,Theon-Greyjoy,4.371193
25,Walder-Frey-(son-of-Merrett),4.184687
24,Walder-Frey-(son-of-Jammos),2.007751
43,Rodrik-Cassel,0.918799
34,Sour-Alyn,0.492669
21,Ramsay-Snow,0.469812
33,Skinner,0.382866
9,Dagmer,0.372420
22,Squint,0.331688
2,Asha-Greyjoy,0.303000


Let's now calculate the intra-community Page Rank for all the communities:

In [115]:
query = """\
CALL algo.pageRank(
  'MATCH (c:Character) WHERE c.community=%d RETURN id(c) as id',
  'MATCH (c:Character)-[rel]->(c2) WHERE c.community=%d AND c2.community =%d RETURN id(c) as source,id(c2) as target, sum(rel.weight) as weight',
  {graph:'cypher', writeProperty: 'communityPageRank'}) 
YIELD nodes 
RETURN count(*)
"""

with driver.session() as session:
    for row in session.run("MATCH (c:Character) WHERE EXISTS(c.community) RETURN DISTINCT c.community AS communityId"):        
        community_id = row["communityId"]
        session.run(query % (community_id, community_id, community_id))
print("Page Ranks calculated")        

Page Ranks calculated


We can now work out who the most influential people are inside and outside a community:

In [116]:
query = """\
MATCH (c:Character)
WHERE exists(c.community)
WITH c ORDER BY c.pageRank DESC
WITH  c.community as cluster, count(*) AS count, collect(c) AS characters
RETURN cluster, count, 
       apoc.coll.reverse(apoc.coll.sortNodes(characters, "pageRank"))[0].name AS overallInfluential,
       apoc.coll.reverse(apoc.coll.sortNodes(characters, "communityPageRank"))[0].name AS communityInfluential
ORDER BY count DESC
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])   

In [117]:
df.sort_values(by=["count"], ascending = False).head(10)

,cluster,communityInfluential,count,overallInfluential
0,760.0,Tywin-Lannister,321,Tyrion-Lannister
1,136.0,Theon-Greyjoy,45,Theon-Greyjoy
2,137.0,Tormund,18,Tormund
3,561.0,Missandei,18,Daenerys-Targaryen
4,556.0,Victarion-Greyjoy,16,Victarion-Greyjoy
5,569.0,Quentyn-Martell,11,Quentyn-Martell
6,143.0,Rhaegar-Frey,10,Theomore
7,301.0,Satin,8,Satin
8,755.0,Quaro,8,Quaro
9,570.0,Obara-Sand,7,Garin-(orphan)
